# Simulating a Multi-Agent AI System for Patient Care Coordination

**Copyright (c) 2026 Shrikara Kaudambady. All rights reserved.**

This notebook simulates a team of specialized AI agents collaborating to manage a new patient's journey. The system is led by a `CareCoordinatorAgent` that orchestrates the workflow by delegating tasks to a `TriageAgent`, `MedicalHistoryAgent`, `InsuranceAgent`, and `SchedulerAgent`. This demonstrates how a complex process can be automated by breaking it down into distinct roles managed by AI.

### 1. The Simulated Healthcare Environment
First, we define our mock databases that the agents will interact with. In a real system, the agents would make API calls to these services.

In [ ]:
class PatientDB:
    """Simulates a database of patient electronic health records (EHR)."""
    def __init__(self):
        self.records = {
            'PID-001': {'name': 'John Doe', 'history': ['hypertension'], 'allergies': ['penicillin'], 'insurance': {'id': 'INS-A1', 'active': True}},
            'PID-002': {'name': 'Jane Smith', 'history': ['diabetes type 2'], 'allergies': [], 'insurance': {'id': 'INS-B2', 'active': False}},
        }
    def get_patient_record(self, patient_id):
        return self.records.get(patient_id)

class DoctorCalendar:
    """Simulates a doctor's appointment calendar."""
    def __init__(self):
        self.slots = {
            '2026-10-26 10:00': 'available',
            '2026-10-26 11:00': 'booked',
            '2026-10-27 14:00': 'available',
        }
    def find_available_slot(self):
        for slot, status in self.slots.items():
            if status == 'available':
                return slot
        return None
    
    def book_slot(self, slot, patient_id):
        if self.slots.get(slot) == 'available':
            self.slots[slot] = f"booked by {patient_id}"
            return True
        return False

### 2. The Specialist AI Agents
Each agent has a specific role and a set of tools (methods) to perform its job.

In [ ]:
class TriageAgent:
    def classify_urgency(self, symptoms):
        print(f"TRIAGE_AGENT: Analyzing symptoms: '{symptoms}'")
        symptoms = symptoms.lower()
        if 'chest pain' in symptoms or 'dizziness' in symptoms or 'breathing difficulty' in symptoms:
            return 'Emergency'
        elif 'fever' in symptoms or 'headache' in symptoms:
            return 'Urgent'
        else:
            return 'Non-Urgent'

class MedicalHistoryAgent:
    def __init__(self, patient_db):
        self.db = patient_db
    def get_summary(self, patient_id):
        print(f"HISTORY_AGENT: Retrieving record for {patient_id}...")
        record = self.db.get_patient_record(patient_id)
        if record:
            return f"Patient has a history of {record['history']} and allergies to {record['allergies']}."
        return "No record found."

class InsuranceAgent:
    def __init__(self, patient_db):
        self.db = patient_db
    def verify_coverage(self, patient_id):
        print(f"INSURANCE_AGENT: Verifying coverage for {patient_id}...")
        record = self.db.get_patient_record(patient_id)
        if record and record['insurance']['active']:
            return True
        return False

class SchedulerAgent:
    def __init__(self, calendar):
        self.calendar = calendar
    def schedule_appointment(self, patient_id):
        print(f"SCHEDULER_AGENT: Searching for an available appointment slot...")
        slot = self.calendar.find_available_slot()
        if slot:
            print(f"SCHEDULER_AGENT: Found available slot at {slot}. Booking now.")
            if self.calendar.book_slot(slot, patient_id):
                return slot
        return None

### 3. The Care Coordinator (Orchestrator) Agent
This is the master agent. It manages the entire workflow, calling the specialist agents in the correct order to process a patient case.

In [ ]:
class CareCoordinatorAgent:
    def __init__(self, triage_agent, history_agent, insurance_agent, scheduler_agent):
        self.triage = triage_agent
        self.history = history_agent
        self.insurance = insurance_agent
        self.scheduler = scheduler_agent
        print("CareCoordinatorAgent is online and ready to manage patient journeys.")
        
    def handle_patient_intake(self, patient_id, symptoms):
        print(f"\n{'='*20} NEW PATIENT CASE: {patient_id} {'='*20}")
        
        # Step 1: Triage
        urgency = self.triage.classify_urgency(symptoms)
        print(f"COORDINATOR: Triage complete. Urgency level: {urgency}")
        if urgency == 'Emergency':
            print("COORDINATOR: This is an emergency. Advising patient to go to the ER immediately. Journey ends.")
            return {"status": "Emergency Referral", "notes": "Patient advised to go to ER."}
        
        # Step 2: Get Medical History
        history_summary = self.history.get_summary(patient_id)
        print(f"COORDINATOR: Received history summary: {history_summary}")

        # Step 3: Verify Insurance
        is_covered = self.insurance.verify_coverage(patient_id)
        print(f"COORDINATOR: Insurance coverage verified. Active: {is_covered}")
        if not is_covered:
            print("COORDINATOR: Patient insurance is not active. Notifying billing department. Journey ends.")
            return {"status": "Insurance Issue", "notes": "Patient has inactive insurance."}
            
        # Step 4: Schedule Appointment
        print("COORDINATOR: Proceeding to schedule appointment.")
        appointment_slot = self.scheduler.schedule_appointment(patient_id)
        if appointment_slot:
            print("COORDINATOR: Appointment successfully booked.")
            final_notes = f"Patient triaged as {urgency}. History: {history_summary} Appointment booked for {appointment_slot}."
            return {"status": "Appointment Booked", "slot": appointment_slot, "notes": final_notes}
        else:
            print("COORDINATOR: No available slots. Notifying admin staff.")
            return {"status": "Scheduling Failed", "notes": "No available appointment slots found."}

### 4. Run the Full Simulation
Let's simulate a few different patient journeys to see the agent team in action.

In [ ]:
# 1. Initialize the environment and all agents
patient_db = PatientDB()
calendar = DoctorCalendar()

triage_agent = TriageAgent()
history_agent = MedicalHistoryAgent(patient_db)
insurance_agent = InsuranceAgent(patient_db)
scheduler_agent = SchedulerAgent(calendar)

coordinator = CareCoordinatorAgent(triage_agent, history_agent, insurance_agent, scheduler_agent)

# 2. Run different scenarios
# Scenario 1: A routine, non-urgent case
result1 = coordinator.handle_patient_intake('PID-001', "I have a sore throat and a slight cough.")
print(f"\n--> FINAL OUTCOME for PID-001: {result1}\n")

# Scenario 2: An emergency case
result2 = coordinator.handle_patient_intake('PID-002', "I have severe chest pain and dizziness.")
print(f"\n--> FINAL OUTCOME for PID-002: {result2}\n")

# Scenario 3: A case with an insurance issue
result3 = coordinator.handle_patient_intake('PID-002', "I need a checkup for my diabetes.")
print(f"\n--> FINAL OUTCOME for PID-002: {result3}")